In [1]:
%matplotlib inline
import numpy as np
import sys
import math
from sklearn.preprocessing import Imputer
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
#import feather
import xgboost as xgb
from sklearn import datasets
import matplotlib

In [2]:
from ROOT import *
from root_pandas import to_root

ImportError: libssl.so.1.0.0: cannot open shared object file: No such file or directory

In [3]:
import json
with open('./C1C1_dict.json') as json_file:
    sign_dict = json.load(json_file)

In [4]:
matplotlib.rcParams.update({'font.size': 20})
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 15
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size

In [5]:
indir = "/scratch3/eirikgr/ML_files/2L0J/newMC16D/"

We read the panda from the h5 files. If both background and signal is in the same file only this is needed:

In [6]:
df = pd.read_hdf(indir+"all_bkg_sig.h5")

In [ ]:
isDATA = True
if isDATA:
    dfdata = pd.read_hdf(indir+"all_data.h5")

If backgrounds and signal are in different files we need to read in both and concatentae them into one big panda:

In [ ]:
eventFile = h5py.File(indir+"allbkg_MVA_ntuples.h5","r")
key = eventFile.keys()[0] #"_".join(h5f.split("_")[-3:-1])
print key
df1 = pd.DataFrame(np.copy(eventFile[key]),columns=eventFile.attrs["var_names"])
#df1 = pd.read_hdf( 'result')

In [ ]:
eventFile = h5py.File(indir+"allsig_MVA_ntuples.h5","r")
key = eventFile.keys()[0] #"_".join(h5f.split("_")[-3:-1])
print key
df2 = pd.DataFrame(np.copy(eventFile[key]),columns=eventFile.attrs["var_names"])

In [ ]:
df = pd.concat([df1,df2])
#del [df1,df2]

In [ ]:
del [df1,df2]

We need some information on the available variables (features), the isSignal labels and the dsids in the sample

In [12]:
# filled below
featurevec = []
df_label = df.isSignal
#print df.columns
#print df.shape
#print df.columns[0]
dsid = np.unique(df.iloc[:,[0]].as_matrix())
#dsid = df.as_matrix(columns=df.columns[0])
print sorted(dsid)
#for col in df
#featurevec = df.index.tolist()
#print featurevec
print "Loaded %i DSIDs" %(len(dsid))
dsid.dtype

[304014.0, 308092.0, 308093.0, 308094.0, 341425.0, 344235.0, 345055.0, 345057.0, 345060.0, 345061.0, 345120.0, 345217.0, 345319.0, 345324.0, 345325.0, 345336.0, 345337.0, 345446.0, 345940.0, 361600.0, 361601.0, 361603.0, 361604.0, 361607.0, 361610.0, 364100.0, 364101.0, 364102.0, 364103.0, 364104.0, 364105.0, 364106.0, 364107.0, 364108.0, 364109.0, 364110.0, 364111.0, 364112.0, 364113.0, 364114.0, 364115.0, 364116.0, 364117.0, 364118.0, 364119.0, 364120.0, 364121.0, 364122.0, 364123.0, 364124.0, 364125.0, 364126.0, 364127.0, 364130.0, 364131.0, 364132.0, 364133.0, 364134.0, 364135.0, 364136.0, 364137.0, 364138.0, 364139.0, 364140.0, 364141.0, 364198.0, 364199.0, 364200.0, 364201.0, 364202.0, 364203.0, 364204.0, 364205.0, 364206.0, 364207.0, 364208.0, 364209.0, 364212.0, 364213.0, 364280.0, 364281.0, 395259.0, 395264.0, 395265.0, 395267.0, 395268.0, 395269.0, 395274.0, 395275.0, 395276.0, 395277.0, 395278.0, 395279.0, 395280.0, 395281.0, 395282.0, 395287.0, 395288.0, 395289.0, 395290.0,

/scratch/eirikgr/miniconda3/envs/testenv/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


dtype('float64')

In [ ]:
is2L2J = True
isMich = False
removeVar = []
featurevec = []
if is2L2J:
    removeVar.append("DSID")
    removeVar.append("RunNumber")
    removeVar.append("EventNumber")
    removeVar.append("eventWeightNoPU")
    removeVar.append("lumiWeight")
    removeVar.append("mu") 
    removeVar.append("eventWeight")
elif isMich:
    removeVar.append("dsid")
    removeVar.append("isSignal") 
    removeVar.append("wgt")
for col in dfdata.columns:
    if col in removeVar: continue
    if "syst_" in col: 
        removeVar.append(col)
        continue
    if col.startswith("Weight"):
        removeVar.append(col)
        continue
    print "Keeping %s" %col
    featurevec.append(col)

In [8]:
is2L2J = True
isMich = False
removeVar = []
if is2L2J:
    removeVar.append("DSID")
    removeVar.append("RunNumber")
    removeVar.append("EventNumber")
    removeVar.append("eventWeightNoPU")
    removeVar.append("lumiWeight")
    removeVar.append("mu") 
    removeVar.append("isSignal") 
    removeVar.append("eventWeight")
    removeVar.append("MCtype")
elif isMich:
    removeVar.append("dsid")
    removeVar.append("isSignal") 
    removeVar.append("wgt")
for col in df.columns:
    if col in removeVar: continue
    if "syst_" in col: 
        removeVar.append(col)
        continue
    if col.startswith("Weight"):
        removeVar.append(col)
        continue
    print "Keeping %s" %col
    featurevec.append(col)

Keeping lept1Pt
Keeping lept2Pt
Keeping lept3Pt
Keeping lept1Eta
Keeping lept2Eta
Keeping lept3Eta
Keeping lept1Phi
Keeping lept2Phi
Keeping lept3Phi
Keeping lept1E
Keeping lept2E
Keeping lept3E
Keeping lept1Flav
Keeping lept2Flav
Keeping lept3Flav
Keeping metPhi
Keeping MET
Keeping metPhi_ele
Keeping MET_ele
Keeping metPhi_mu
Keeping MET_mu
Keeping metPhi_jet
Keeping MET_jet
Keeping metPhi_gam
Keeping MET_gam
Keeping metPhi_soft
Keeping MET_soft
Keeping METsig
Keeping nLeps
Keeping pbllmod
Keeping L2Mll
Keeping L2MT2
Keeping L2isEMU
Keeping L2isEE
Keeping L2isMUMU
Keeping L2nCentralLightJet
Keeping L2nCentralBJet
Keeping L2nForwardJet
Keeping L2nCentralLightJet30
Keeping L2nCentralBJet30
Keeping L2nForwardJet30
Keeping L2nCentralLightJet40
Keeping L2nCentralBJet40
Keeping L2nForwardJet40
Keeping L2nCentralLightJet50
Keeping L2nCentralBJet50
Keeping L2nForwardJet50
Keeping L2nCentralLightJet60
Keeping L2nCentralBJet60
Keeping L2nForwardJet60
Keeping jet1Pt
Keeping jet2Pt
Keeping jet3Pt

In [9]:
rBackgroundEvents = 0.5
rSignalEvents = 0.5

In [10]:
indexNamesArr = df.index.values
indexNamesArr[1]

1

In [13]:
nx = 0
startnewfile = True
nfile = 0
for ids in dsid:
    print "Doing %i/%i" %(nx,len(dsid))
    newdf = df.loc[df['DSID'] == ids]
    print "DSID %.0f has %i events" %(ids,newdf.shape[0])
    if startnewfile:
        X_train = newdf.sample(frac=rBackgroundEvents)
        X_test  = newdf.drop(X_train.index.values)
        startnewfile = False
    else:
        midl = newdf.sample(frac=rBackgroundEvents)
        X_train = pd.concat([X_train,midl],axis=0)
        X_test  = pd.concat([X_test, newdf.drop(midl.index.values)],axis=0)
        del [midl]
    del [newdf]
    nx += 1
    if nx%10 == 0:
        path = indir+"/testing_bkg_sig_%i.h5"%nfile
        print "Writing to file %s" %path
        X_test.to_hdf(path,key='result', mode='w')
        path = indir+"/training_bkg_sig_%i.h5"%nfile
        print "Writing to file %s" %path
        X_train.to_hdf(path,key='result', mode='w')
        nfile += 1
        del [X_test]
        del [X_train]
        startnewfile = True
path = indir+"/testing_bkg_sig_%i.h5"%nfile
print "Writing to file %s" %path
X_test.to_hdf(path,key='result', mode='w')
path = indir+"/training_bkg_sig_%i.h5"%nfile
print "Writing to file %s" %path
X_train.to_hdf(path,key='result', mode='w')
del [X_test]
del [X_train]

Doing 0/383
DSID 304014 has 2412 events
Doing 1/383
DSID 308092 has 11572 events
Doing 2/383
DSID 308093 has 14648 events
Doing 3/383
DSID 308094 has 1 events
Doing 4/383
DSID 341425 has 200 events
Doing 5/383
DSID 344235 has 29083 events
Doing 6/383
DSID 345055 has 294451 events
Doing 7/383
DSID 345057 has 64434 events
Doing 8/383
DSID 345060 has 18273 events
Doing 9/383
DSID 345061 has 231 events
Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//testing_bkg_sig_0.h5
Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//training_bkg_sig_0.h5
Doing 10/383
DSID 345120 has 60 events
Doing 11/383
DSID 345217 has 2955 events
Doing 12/383
DSID 345319 has 652 events
Doing 13/383
DSID 345324 has 13946 events
Doing 14/383
DSID 345325 has 2898 events
Doing 15/383
DSID 345336 has 3976 events
Doing 16/383
DSID 345337 has 36429 events
Doing 17/383
DSID 345446 has 7939 events
Doing 18/383
DSID 345940 has 198032 events
Doing 19/383
DSID 361600 has 244143 events
Writing to file /scratch3/

DSID 395898 has 10481 events
Doing 146/383
DSID 395908 has 3015 events
Doing 147/383
DSID 395909 has 5424 events
Doing 148/383
DSID 395910 has 9547 events
Doing 149/383
DSID 395920 has 2044 events
Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//testing_bkg_sig_14.h5
Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//training_bkg_sig_14.h5
Doing 150/383
DSID 395930 has 1660 events
Doing 151/383
DSID 395931 has 2757 events
Doing 152/383
DSID 395946 has 1012 events
Doing 153/383
DSID 395947 has 1732 events
Doing 154/383
DSID 395948 has 3291 events
Doing 155/383
DSID 395949 has 4308 events
Doing 156/383
DSID 395962 has 217 events
Doing 157/383
DSID 395963 has 683 events
Doing 158/383
DSID 395964 has 902 events
Doing 159/383
DSID 395965 has 1001 events
Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//testing_bkg_sig_15.h5
Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//training_bkg_sig_15.h5
Doing 160/383
DSID 395969 has 2 events
Doing 161/383
DSID 395970

Doing 290/383
DSID 397117 has 4549 events
Doing 291/383
DSID 397118 has 2100 events
Doing 292/383
DSID 397119 has 1536 events
Doing 293/383
DSID 397120 has 3132 events
Doing 294/383
DSID 397121 has 2309 events
Doing 295/383
DSID 397122 has 1093 events
Doing 296/383
DSID 397123 has 2562 events
Doing 297/383
DSID 397124 has 917 events
Doing 298/383
DSID 397125 has 860 events
Doing 299/383
DSID 397126 has 862 events
Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//testing_bkg_sig_29.h5
Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//training_bkg_sig_29.h5
Doing 300/383
DSID 397127 has 1661 events
Doing 301/383
DSID 397128 has 1285 events
Doing 302/383
DSID 397129 has 500 events
Doing 303/383
DSID 397130 has 1375 events
Doing 304/383
DSID 397131 has 477 events
Doing 305/383
DSID 397132 has 947 events
Doing 306/383
DSID 397133 has 933 events
Doing 307/383
DSID 397134 has 1757 events
Doing 308/383
DSID 397135 has 1682 events
Doing 309/383
DSID 397136 has 1341 events
Writin

In [14]:
path = indir+"/testing_bkg_sig_%i.h5"%nfile
print "Writing to file %s" %path
X_test.to_hdf(path,key='result', mode='w')
path = indir+"/training_bkg_sig_%i.h5"%nfile
print "Writing to file %s" %path
X_train.to_hdf(path,key='result', mode='w')

Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//testing_bkg_sig_38.h5
Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//training_bkg_sig_38.h5


In [15]:
del [X_test]
del [X_train]

Reading back in the testing sample

In [16]:
nfile = 0
for f in sorted(listdir(indir)):
    if not "testing_bkg_sig_" in f: continue
    print "Reading in file %s" %f
    if nfile == 0: X_test = pd.read_hdf(indir+"/"+f)
    else: X_test  = pd.concat([X_test,pd.read_hdf(indir+"/"+f)],axis=0)
    nfile += 1

Reading in file testing_bkg_sig_15.h5
Reading in file testing_bkg_sig_17.h5
Reading in file testing_bkg_sig_35.h5
Reading in file testing_bkg_sig_19.h5
Reading in file testing_bkg_sig_28.h5
Reading in file testing_bkg_sig_12.h5
Reading in file testing_bkg_sig_3.h5
Reading in file testing_bkg_sig_22.h5
Reading in file testing_bkg_sig_2.h5
Reading in file testing_bkg_sig_26.h5
Reading in file testing_bkg_sig_20.h5
Reading in file testing_bkg_sig_16.h5
Reading in file testing_bkg_sig_25.h5
Reading in file testing_bkg_sig_1.h5
Reading in file testing_bkg_sig_14.h5
Reading in file testing_bkg_sig_27.h5
Reading in file testing_bkg_sig_8.h5
Reading in file testing_bkg_sig_18.h5
Reading in file testing_bkg_sig_24.h5
Reading in file testing_bkg_sig_33.h5
Reading in file testing_bkg_sig_11.h5
Reading in file testing_bkg_sig_10.h5
Reading in file testing_bkg_sig_0.h5
Reading in file testing_bkg_sig_9.h5
Reading in file testing_bkg_sig_34.h5
Reading in file testing_bkg_sig_31.h5
Reading in file te

... and the training sample

In [19]:
nfile = 0
for f in sorted(listdir(indir)):
    if not "training_bkg_sig_" in f: continue
    print "Reading in file %s" %f
    if nfile == 0: X_train = pd.read_hdf(indir+"/"+f)
    else: X_train  = pd.concat([X_train,pd.read_hdf(indir+"/"+f)],axis=0)
    nfile += 1

Reading in file training_bkg_sig_0.h5
Reading in file training_bkg_sig_1.h5
Reading in file training_bkg_sig_10.h5
Reading in file training_bkg_sig_11.h5
Reading in file training_bkg_sig_12.h5
Reading in file training_bkg_sig_13.h5
Reading in file training_bkg_sig_14.h5
Reading in file training_bkg_sig_15.h5
Reading in file training_bkg_sig_16.h5
Reading in file training_bkg_sig_17.h5
Reading in file training_bkg_sig_18.h5
Reading in file training_bkg_sig_19.h5
Reading in file training_bkg_sig_2.h5
Reading in file training_bkg_sig_20.h5
Reading in file training_bkg_sig_21.h5
Reading in file training_bkg_sig_22.h5
Reading in file training_bkg_sig_23.h5
Reading in file training_bkg_sig_24.h5
Reading in file training_bkg_sig_25.h5
Reading in file training_bkg_sig_26.h5
Reading in file training_bkg_sig_27.h5
Reading in file training_bkg_sig_28.h5
Reading in file training_bkg_sig_29.h5
Reading in file training_bkg_sig_3.h5
Reading in file training_bkg_sig_30.h5
Reading in file training_bkg_

In [20]:
print "%i events in testing sample, %i variables" %(X_test.shape[0],X_test.shape[1])
print "%i events in training sample, %i variables"%(X_train.shape[0],X_train.shape[1])
print "Will remove %i variables " %len(removeVar)

3557451 events in testing sample, 126 variables
3557645 events in training sample, 126 variables
Will remove 67 variables 


In [21]:
path = indir+"/testing_bkg_sig_ALL.h5"
print "Writing to file %s" %path
X_test.to_hdf(path,key='result', mode='w')
path = indir+"/training_bkg_sig_ALL.h5"
print "Writing to file %s" %path
X_train.to_hdf(path,key='result', mode='w')

Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//testing_bkg_sig_ALL.h5
Writing to file /scratch3/eirikgr/ML_files/2L0J/newMC16D//training_bkg_sig_ALL.h5


Now we will make the testing and training arrays. For memory reasons we delete the full array afterwards (since the same infomration now is sotred in the two training/testing arrays.

In [22]:
Y_test_array = np.array(X_test.isSignal)
X_test_array = np.array(X_test.drop(removeVar,axis=1))
W_test_array = np.array(X_test.eventWeight)
ID_test_array = np.array(X_test.DSID)
MC_test_array = np.array(X_test.MCtype)
#del [X_test]

In [23]:
Y_train_array = np.array(X_train.isSignal)
X_train_array = np.array(X_train.drop(removeVar,axis=1))
W_train_array = np.array(X_train.eventWeight)
ID_train_array = np.array(X_train.DSID)
MC_train_array = np.array(X_train.MCtype)
del [X_train]
#W_test_array = np.array(X_test.wgt) * float(X_test.shape[0])

In [ ]:
if isDATA:
    X_data_array = np.array(dfdata.drop(removeVar,axis=1))

In [ ]:
#X_test#
#newdf = X_test[:][0:int(X_test.shape[0]/2.)]
#print X_test.shape
#print newdf.shape
#X_test_array = np.array(newdf.drop(removeVar,axis=1))
#newdf = X_test[:][int(X_test.shape[0]/2.):]
#print X_test.shape
#print newdf.shape
#midl = np.array(newdf.drop(removeVar,axis=1))
#X_test_array = np.concatenate((X_test_array,midl),axis=0)


#X_train_array = np.array(X_train.drop(removeVar,axis=1))

In [24]:
print "%i events in testing sample, %i variables" %(X_test_array.shape[0],X_test_array.shape[1])
print "%i events in training sample, %i variables"%(X_train_array.shape[0],X_train_array.shape[1])

3557451 events in testing sample, 59 variables
3557645 events in training sample, 59 variables


We need the sum of weights since the testing and training sample need to be scaled up with the sum of weights in the training and testing sample, respectively. We also need to get the sum of weights for signal and background seperately since the dataset is very unbalanced and so must be accounted for in the BDT.

In [25]:
sum_wsig_train = sum( W_train_array[i] for i in range(len(Y_train_array)) if Y_train_array[i] == 1.0  )#
sum_wbkg_train = sum( W_train_array[i] for i in range(len(Y_train_array)) if Y_train_array[i] == 0.0  )

sum_w_train = sum( W_train_array[i] for i in range(len(Y_train_array)))
sum_w_test = sum( W_test_array[i] for i in range(len(Y_test_array)))

# scaling the weights of the testing and training sample so that we account 
# for the fact that we've split the full set into two
W_train_array = W_train_array * (sum_w_test+sum_w_train)/sum_w_train
W_test_array = W_test_array * (sum_w_test+sum_w_train)/sum_w_test

In [26]:
print "Sum of weights for training                   = %.2f" %sum_w_train
print "Sum of weights for testing                    = %.2f" %sum_w_test
print "Fraction of events for training               = %.2f" %float(sum_w_test/(sum_w_train+sum_w_test))
print "Sum of weights for signal (training)          = %.2f" %sum_wsig_train
print "Sum of weights for backgrounds (training)     = %.2f" %sum_wbkg_train
print "Fraction of signal events in training sample  = %.2f" %(sum_wsig_train/(sum_wsig_train+sum_wbkg_train))

Sum of weights for training                   = 198961.37
Sum of weights for testing                    = 199299.35
Fraction of events for training               = 0.50
Sum of weights for signal (training)          = 31603.04
Sum of weights for backgrounds (training)     = 167358.32
Fraction of signal events in training sample  = 0.16


In [ ]:
# Set the size of each portion of data
#test_size = 50000
# Convert to numpy arrays (input)
# Separate into data, weights, target
#Y = np.array(df_label)
#X = np.array(df.drop(removeVar,axis=1)) # Remove variables that should NOT be trained on
#if is2L2J:
#    W = np.array(df.eventWeight) * float(test_size) / len(Y) # Ensure the weights are scaled to the size of the test portion
#elif isMich:
#    W = np.array(df.wgt) * float(test_size) / len(Y) # Ensure the weights are scaled to the size of the test portion

In [ ]:
# get the featurevector
#featurevec = df.drop(removeVar,axis=1).columns
#print featurevec[0]

If you are in memory trouble you could delete this

In [27]:
del [df]

In [ ]:
#print df.columns[1:]
#dsid = df.as_matrix(columns=df.columns[1:])

In [ ]:
# Build training, validation & evaluation samples (test_size events for each of signal and background per sample)
#size = test_size
#X_train = np.concatenate((X[Y==0][0:size],X[Y==1][0:size]), axis=0)
#w_train = np.concatenate((W[Y==0][0:size],W[Y==1][0:size]), axis=0)
#y_train = np.concatenate((Y[Y==0][0:size],Y[Y==1][0:size]), axis=0)
#X_valid = np.concatenate((X[Y==0][size:size*2],X[Y==1][size:size*2]), axis=0)
#y_valid = np.concatenate((Y[Y==0][size:size*2],Y[Y==1][size:size*2]), axis=0)
#X_eval = np.concatenate((X[Y==0][size*2:size*3],X[Y==1][size*2:size*3]), axis=0)
#y_eval = np.concatenate((Y[Y==0][size*2:size*3],Y[Y==1][size*2:size*3]), axis=0)
# Get the sums of the weights for signal and background events 
#sum_wsig = sum( w_train[i] for i in range(len(y_train)) if y_train[i] == 1.0  )
#sum_wbkg = sum( w_train[i] for i in range(len(y_train)) if y_train[i] == 0.0  )

In [ ]:
#dtrain = xgb.DMatrix(X_train,label=y_train,weight=w_train,feature_names=featurevec)
#print (sum_wbkg_train+sum_wsig_train)/sum_wsig_train

In [ ]:
# Hyperparameter scan
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import  balanced_accuracy_score, make_scorer

balanced_scorer = make_scorer(balanced_accuracy_score)

xgb_model = xgb.XGBClassifier()

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4),
    "scale_pos_weight":[(sum_wbkg_train+sum_wsig_train)/sum_wsig_train]
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, 
                            n_iter=200, cv=3, verbose=1, n_jobs=4, scoring=balanced_scorer, 
                            return_train_score=True,pre_dispatch=8)

search.fit(X_train_array,Y_train_array,W_train_array)

report_best_scores(search.cv_results_, 1)

In [29]:
# Set up the BDT classifier
xgbclassifier = xgb.XGBClassifier(
    max_depth=3, 
    n_estimators=120,
    learning_rate=0.1,
    n_jobs=6,
    scale_pos_weight=(sum_wbkg_train+sum_wsig_train)/sum_wsig_train,
    objective='binary:logistic') 
xgbclassifier.fit(X_train_array,Y_train_array,W_train_array) 

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=120,
       n_jobs=6, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=6.295639296005678,
       seed=None, silent=True, subsample=1)

In [30]:
# Test the BDT performance using the validation dataset
y_pred = xgbclassifier.predict( X_test_array ) # The actual signal/background predictions. Note that we don't actually use them
y_pred_prob = xgbclassifier.predict_proba( X_test_array ) # The BDT outputs for each event

In [ ]:
if isDATA:
    y_pred_data = xgbclassifier.predict( X_data_array ) # The actual signal/background predictions. Note that we don't actually use them
    y_pred_prob_data = xgbclassifier.predict_proba( X_data_array ) # The BDT outputs for each event

In [ ]:
mctype_unique = np.unique(MC_test_array)#np.unique(df_mctype.iloc[:].as_matrix())

In [ ]:
print mctype_unique

Plot the output from XGBoost for background and signal

In [ ]:
#ar = y_pred_prob[:,1][MC_test_array=="Diboson_PowHeg_WW"]
#x = np.linspace(0,1,101)
#df_bkg_test  = pd.Series(y_pred_prob[:,1])
X_test["predprob"] = pd.Series(y_pred_prob[:,1],index=X_test.index,dtype='float')
X_test["eventWeight"] *= (sum_w_test+sum_w_train)/sum_w_test



In [ ]:
if isDATA:
    dfdata["predprob"] = pd.Series(y_pred_prob_data[:,1],index=dfdata.index,dtype='float')

In [ ]:
X_test.to_root(indir+'/XGBoost_default_test.root', key='XGB')
#df2 = X_test.groupby(['MCtype', 'predprob'])['predprob'].count().unstack('MCtype').fillna(0)

In [ ]:
if isDATA:
    dfdata.to_root(indir+'/XGBoost_default_data.root', key='XGB')

In [ ]:
del [df2]

In [ ]:
df2.plot(kind='bar',stacked=True)

In [ ]:
#  histogram of the BDT outputs

p1 = plt.bar(x,y_pred_prob[:,1][MC_test_array=="Diboson_PowHeg_WW"])
p2 = plt.bar(x,y_pred_prob[:,1][MC_test_array=="Diboson_PowHeg_WZ"],
             bottom=y_pred_prob[:,1][MC_test_array=="Diboson_PowHeg_WW"])
#n_bkg, bins_bkg, patches = plt.hist([y_pred_prob[:,1][MC_test_array=="Diboson_PowHeg_WW"],
                                    # y_pred_prob[:,1][MC_test_array=="Diboson_PowHeg_WZ"]], 
                                    #weights=[W_test_array[MC_test_array=="Diboson_PowHeg_WW"],
                                   #          W_test_array[MC_test_array=="Diboson_PowHeg_WZ"]], 
                                   # bins=200, normed=0, facecolor=['blue','green'], 
                                    #label=["WW","WZ"], histtype='bar', stacked=True)
#n_bkg, bins_bkg, patches = plt.hist(y_pred_prob[:,1][MC_test_array=="Diboson_PowHeg_WZ"], 
                                    #weights=W_test_array[MC_test_array=="Diboson_PowHeg_WZ"], 
                                    #bins=200, normed=0, facecolor='green', 
                                    #label="WZ", histtype='barstacked')
#n_sig, bins_sig, patches = plt.hist(y_pred_prob[:,1][Y_test_array==1], weights=W_test_array[Y_test_array==1], bins=200, normed=0, facecolor='red', alpha=0.2, label="Signal")

plt.xlabel('XGBoost output')
plt.ylabel('Events')
plt.title('XGBoost output, SUSY C1C1->WW dataset, validation data')
plt.grid(True)
plt.legend()
plt.yscale("log")
plt.show()

In [ ]:
execfile("plot_confusion_matrix.py")
import plot_confusion_matrix as pcm

In [ ]:
from sklearn.utils.multiclass import unique_labels
unique_labels(Y_test_array, y_pred)

In [ ]:
pcm.plot_confusion_matrix(Y_test_array, y_pred,np.array(["background","signal"]),normalize=False)

In [ ]:
signaldid = []
for did in dsid:
    if did >= 390000 and did < 400000: signaldid.append(did)    
x_did = np.zeros(len(signaldid))
y_did = np.zeros(len(signaldid))

In [ ]:
from ROOT import TH1F, Double, RooStats, TFile
h_predprob_sig = TH1F("h_predprob_sig","h_predprob_sig",200,0,1)
h_predprob_bkg = TH1F("h_predprob_bkg","h_predprob_bkg",200,0,1)
for i in range(len(y_pred_prob[:,1])):
    if Y_test_array[i] == 1: 
        h_predprob_sig.Fill(y_pred_prob[:,1][i],W_test_array[i])
        #if y_pred_prob[:,1][i] > 0.99:
        #    print sign_dict["%.0f"%ID_test_array[i]]
    elif Y_test_array[i] == 0: h_predprob_bkg.Fill(y_pred_prob[:,1][i],W_test_array[i]) 

In [ ]:
from math import e, pi, isinf
max_sign = -999
max_nsig = -999
max_nbkg = -999
cutval = -1
cutbin = -1
y_sign = np.zeros(h_predprob_sig.GetNbinsX())
x_sign = np.zeros(h_predprob_sig.GetNbinsX())
ni = 0
err_sig = Double(0)
err_bkg = Double(0)
for i in range(1,h_predprob_sig.GetNbinsX()+1):
    nsig = h_predprob_sig.IntegralAndError(i,h_predprob_sig.GetNbinsX()+1,err_sig)
    nbkg = h_predprob_bkg.IntegralAndError(i,h_predprob_bkg.GetNbinsX()+1,err_bkg)
    sign = RooStats.NumberCountingUtils.BinomialExpZ(nsig,nbkg,0.2)
    y_sign[i-1] = sign
    x_sign[i-1] = h_predprob_sig.GetBinLowEdge(i)
    if nbkg <= 0: continue
    if nsig <= 0 : continue
    if isinf(sign): continue
    if sign > max_sign:
        max_sign = sign
        max_nbkg = nbkg
        max_nsig = nsig
        cutval = h_predprob_sig.GetBinLowEdge(i)
        cutbin = i
#for i in range(len(bins_sig)-1):
#    print bins_sig[i],bins_bkg[i]
#    nbkg = np.sum(n_bkg[i:])
##    nsig = np.sum(n_sig[i:])
#    sign = RooStats.NumberCountingUtils.BinomialExpZ(nsig,nbkg,0.2)
#    if i%10 == 0: 
#        y_sign[ni] = sign
#        x_sign[ni] = bins_sig[i]
#        ni += 1
#    if nbkg <= 0: continue
#    if nsig <= 0 : continue
#    print sign,nbkg,nsig
#    if sign > max_sign:
#        max_sign = sign
#        max_nbkg = nbkg
#        max_nsig = nsig
#        cutval = bins_sig[i]
    #h_sig.AddBinContent(i+1,sign)

In [ ]:
plt.plot(x_sign,y_sign)

In [ ]:
f = TFile("./test.root","RECREATE")
f.cd()
h_predprob_sig.Write()
h_predprob_bkg.Write()
f.Close()
print max_sign,max_nbkg,max_nsig,cutval,cutbin
sign = RooStats.NumberCountingUtils.BinomialExpZ(max_nsig,max_nbkg,0.2)
print sign

In [ ]:
W_test_array

In [ ]:
h_predprob_sig_all = {}
for i in range(len(y_pred_prob[:,1])):
    if i%1000000 == 0: print "Doing %i/%i" %(i,len(y_pred_prob[:,1]))
    if ID_test_array[i] in signaldid:
        key = ID_test_array[i]
        if not ID_test_array[i] in h_predprob_sig_all.keys():
            h_predprob_sig_all[key] = TH1F("h_predprob_sig_all_%.0f"%key,
                                           "h_predprob_sig_all_%.0f"%key,200,0,1)
        h_predprob_sig_all[key].Fill(y_pred_prob[:,1][i],W_test_array[i])
        if Y_test_array[i] != 1:
            print "ERROR \t Event is signal (%i) with dsid = %.0f" %(Y_test_array[i],ID_test_array[i])
        
#print sign_dict["%.0f"%signaldid[0]]

In [ ]:
#print "%20s %20s %20s %20s" %("split",)
print "Number of backgrpound events %.2f" %max_nbkg
nbkg = max_nbkg
cutval = 0.96
print "Cutting at value %.3f in bin %i" %(cutval,h_predprob_bkg.FindBin(cutval))
nbkg = h_predprob_bkg.IntegralAndError(h_predprob_bkg.FindBin(cutval),h_predprob_bkg.GetNbinsX()+1,err_bkg)
outfil = open("bestSR.txt","w")
for did in h_predprob_sig_all.keys():
    if not "%.0f"%did in sign_dict.keys(): continue
    nsig = h_predprob_sig_all[did].IntegralAndError(h_predprob_sig_all[did].FindBin(cutval),h_predprob_sig_all[did].GetNbinsX()+1,err_sig)
    sign = RooStats.NumberCountingUtils.BinomialExpZ(nsig,nbkg,0.1)
    if sign > 1.64:
        print sign_dict["%.0f"%did], nbkg, nsig, sign 
    outfil.write("%20s %.2f %.2f %.2f \n" %(sign_dict["%.0f"%did], nbkg, nsig, sign))
outfil.close()


In [ ]:
from sklearn.metrics import roc_curve,auc
fpr, tpr, thresholds = roc_curve(Y_test_array,y_pred_prob[:,1], pos_label=1)
roc_auc = auc(fpr,tpr)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for xgBoost on SUSY C1C1->WW dataset')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Plot variable importance
ax = xgb.plot_importance(xgbclassifier)
# convert the features (f1,f2 etc. into the variable names)
labels = []
for lb in ax.get_yticklabels():
    idx = int(lb.get_text().replace("f",""))
    labels.append(featurevec[idx])
ax.set_yticklabels(labels)
#fig_size[0] = 5
#fig_size[1] = 10
#ax.set_yticklabels(featurevec, minor=False)
#plt.rcParams["figure.figsize"] = fig_size
yticks = ax.get_yticks()
ax.set_yticks
plt.show()